In [1]:
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV 
with open('./traindata.txt', 'rb') as f:
    X = pickle.load(f)

In [2]:
X.shape

(15000, 45)

In [3]:
X.head()

,err_1,err_2,err_3,err_4,err_5,err_7,err_8,err_9,err_12,err_13,...,23_ac,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_12
0,0.0,0.0,8.0,104.0,0.0,1.0,0.0,0.0,16.0,1.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0
1,0.0,0.0,0.0,0.0,48.0,1.0,0.0,0.0,11.0,1.0,...,625.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,131.0,1.0,1.0,0.0,0.0,14.0,1.0,...,0.0,0.0,1.0,5.0,44.0,44.0,0.0,1.0,8.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,19.0,3.0,...,5.0,0.0,0.0,2.0,87.0,87.0,0.0,0.0,3.0,0.0


In [4]:
X = X.fillna(0)

In [5]:
train_problem_data = pd.read_csv('./train_problem_data.csv',thousands = ',')

In [6]:
df_tpd = train_problem_data.drop_duplicates()
problem_list = list(df_tpd['user_id'].unique())
y = np.zeros(15000,)
for pi in problem_list:
    y[pi-10000] = 1

In [7]:
y = pd.Series(y)

In [8]:
xgb.__version__

'1.3.3'

In [9]:
def modelfit(alg, X,y,useTrainCV=True, cv_folds=5, early_stopping_rounds=100):
   
    # get new n_estimator
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X.values, label=y.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='error', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(alg)
    
    # Fit the algorithm on the data
    alg.fit(X, y, eval_metric='error')
        
    #Predict training set:
    dtrain_predictions = alg.predict(X)
    dtrain_predprob = alg.predict_proba(X)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Training Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))

In [10]:

xgb1 = xgb.XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=2019
)
modelfit(xgb1, X, y)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, gamma=0, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=None, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=93, n_jobs=None, nthread=-1, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=1, seed=2019, subsample=0.8, tree_method=None,
              validate_parameters=None, verbosity=None)


C:\Users\Bbarroo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Model Report
Training Accuracy : 0.82


In [12]:
param_test1 = {
 'max_depth':range(3,10,3),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019),
param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(X,y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:  9.6min remaining:   26.7s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  9.8min finished
C:\Users\Bbarroo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\Bbarroo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when construct

[18:28:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([21.35819716, 22.30600057, 34.85839987, 74.28342466, 57.13467846,
         49.51745863, 76.17239904, 66.09039941, 52.86200013]),
  'std_fit_time': array([ 0.76633731,  0.06252392, 10.43334478,  3.94147459,  1.38294829,
          2.42804461,  3.2654055 ,  0.36847713,  8.87899072]),
  'mean_score_time': array([0.03640122, 0.04020233, 0.07880335, 0.13899951, 0.06200137,
         0.06860137, 0.07360015, 0.06700044, 0.06280036]),
  'std_score_time': array([0.01369202, 0.00538492, 0.04230955, 0.06726985, 0.01565965,
         0.02009358, 0.01546161, 0.01291586, 0.0141053 ]),
  'param_max_depth': masked_array(data=[3, 3, 3, 6, 6, 6, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[1, 3, 5, 1, 3, 5, 1, 3, 5],
               mask=[False, False, False, False, False, False, False, False,
                    

In [14]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test2, scoring='accuracy', n_jobs=-1, cv=5)
gsearch2.fit(X,y)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

C:\Users\Bbarroo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:31:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { thread } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:31:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([23.31799469, 25.24119554, 24.19480042, 26.62099557, 20.06039896]),
  'std_fit_time': array([1.05730567, 0.17198036, 0.79262582, 1.56477586, 6.2525537 ]),
  'mean_score_time': array([0.04440084, 0.05020275, 0.04119816, 0.05300131, 0.0317996 ]),
  'std_score_time': array([0.01400913, 0.00765241, 0.00803744, 0.01492609, 0.00598106]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'split0_test_score': array([0.76266667, 0.76466667, 0.76766667, 0.76366667, 0.76266667]),
  'split1_test_score': array([0.768     , 0.77      , 0.769     , 0.76466667, 0.76766667]),
  'split2_test_score': array([0.78933333, 0.79066667, 0.79333333, 0.78966667, 0.792     ]),
  'split3_test_score': array([0.77566667, 0.77733333, 0.78      , 0.77833333, 0.781   

In [15]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test3, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch3.fit(X,y)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  8.9min finished
C:\Users\Bbarroo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\Bbarroo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future

[18:40:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { thread } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:40:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([24.7044003 , 34.40819821, 33.00499988, 28.76586595, 26.04179878,
         24.72200046, 23.76079798, 24.04860053, 25.4868001 , 24.69819827,
         24.16239953, 23.11480002, 28.01219802, 29.11159892, 29.00959973,
         24.61099811]),
  'std_fit_time': array([2.13088055, 8.9285082 , 8.30765197, 1.48376689, 0.13681932,
         1.02232117, 0.28831146, 0.16614568, 0.19083873, 0.05689407,
         0.27073765, 0.24579439, 0.31623064, 2.46899372, 2.31309579,
         1.06214544]),
  'mean_score_time': array([0.07919993, 0.0834013 , 0.093999  , 0.07100096, 0.04060035,
         0.05659971, 0.04400234, 0.06079979, 0.03840055, 0.0406013 ,
         0.04160123, 0.05199904, 0.06500058, 0.04519925, 0.05020123,
         0.0394032 ]),
  'std_score_time': array([0.06757643, 0.0464695 , 0.04449609, 0.04144288, 0.00755164,
         0.02128586, 0.00901091, 0.01941449, 0.00873115, 0.00637431,
         0.00588459, 0.0099613 , 0.02186304, 0.01462179, 0.01603067,
         0.010723

In [16]:
param_test4 = {
 'subsample':[i/100.0 for i in range(40,80)],
}
gsearch4 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.6, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test4, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch4.fit(X,y)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 18

[19:01:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { thread } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:01:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([24.24120011, 24.9148026 , 26.19079938, 24.02959681, 24.39399838,
         23.48159919, 23.72039847, 23.90999889, 24.38339944, 24.40819674,
         24.80939951, 25.10939846, 28.18220873, 27.95523658, 30.39121318,
         29.40919933, 24.41919942, 24.90639729, 24.09739952, 25.01160207,
         23.82719741, 23.89660106, 23.90479922, 24.02439861, 23.7699995 ,
         24.71460104, 24.65060034, 24.81420059, 25.52050858, 24.27666035,
         23.41543622, 23.22059932, 23.8400125 , 23.74750218, 23.00711122,
         23.03840022, 22.59502134, 22.6727993 , 22.96159987, 22.17559819]),
  'std_fit_time': array([0.16012537, 0.7376974 , 0.38798716, 1.10955281, 0.49127063,
         0.21595899, 0.09318372, 0.10836407, 0.18091881, 0.144737  ,
         0.38189032, 0.20283214, 0.66294457, 1.46186516, 0.59766463,
         0.85174539, 0.68318806, 0.85682071, 0.14313979, 0.35600483,
         0.08655813, 0.06484946, 0.13779375, 0.03517061, 0.52070919,
         0.15617577, 0.05945

In [17]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.67, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  thread=-1, 
                                                  scale_pos_weight=1,
                                                  seed=2019), 
                        param_grid = param_test5, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch5.fit(X,y)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   52.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  2.4min remaining:   27.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.5min finished
C:\Users\Bbarroo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\Bbarroo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  w

[19:03:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { thread } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:03:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([26.88879795, 25.00404348, 26.58636374, 24.24770765, 12.21170139]),
  'std_fit_time': array([0.97056106, 0.17142978, 0.95559192, 1.65888166, 3.02625219]),
  'mean_score_time': array([0.0590023 , 0.04700098, 0.03260264, 0.0271997 , 0.03000555]),
  'std_score_time': array([0.02020915, 0.01021793, 0.00697476, 0.00453559, 0.01160179]),
  'param_reg_alpha': masked_array(data=[1e-05, 0.01, 0.1, 1, 100],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'reg_alpha': 1e-05},
   {'reg_alpha': 0.01},
   {'reg_alpha': 0.1},
   {'reg_alpha': 1},
   {'reg_alpha': 100}],
  'split0_test_score': array([0.763     , 0.76      , 0.75866667, 0.76766667, 0.76033333]),
  'split1_test_score': array([0.76366667, 0.76666667, 0.768     , 0.76633333, 0.76066667]),
  'split2_test_score': array([0.78733333, 0.785     , 0.78966667, 0.78466667, 0.77133333]),
  'split3_test_score': array([0.77966667, 0.77833333, 0.779 

In [18]:
with open('./testdata.txt', 'rb') as f:
    testdata = pickle.load(f)
testdata.fillna(0)

,err_1,err_2,err_3,err_4,err_5,err_7,err_8,err_9,err_12,err_13,...,23_ac,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_12
0,0.0,0.0,0.0,0.0,44.0,1.0,0.0,0.0,16.0,0.0,...,4.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,2.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,0.0,0.0,16.0,4.0,...,0.0,0.0,0.0,1287.0,0.0,0.0,0.0,0.0,2556.0,0.0
2,0.0,0.0,0.0,0.0,41.0,1.0,0.0,0.0,16.0,0.0,...,6.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0,49.0,0.0
3,0.0,0.0,0.0,0.0,73.0,2.0,0.0,0.0,5.0,1.0,...,0.0,0.0,0.0,4.0,93.0,93.0,0.0,0.0,6.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,0.0,0.0,12.0,1.0,...,3.0,0.0,0.0,2.0,24.0,24.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,0.0,0.0,19.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
14995,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,15.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,1.0,...,11.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,9.0,0.0
14997,0.0,0.0,0.0,1.0,4.0,2.0,0.0,0.0,16.0,0.0,...,20.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,0.0


In [24]:
seeds = [2015,2016,2017,2018,2019]
test_pred = []
for seed in seeds:
    xgb1 = xgb.XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=3,
        min_child_weight=1,
        gamma=0.2,
        reg_alpha=0.1,
        subsample=0.72,
        colsample_bytree=0.9,
        objective= 'binary:logistic',
        nthread=-1,
        scale_pos_weight=1,
        seed=seed
    )
    modelfit(xgb1,X,y)
    if len(test_pred) == 0:
        test_pred = xgb1.predict_proba(testdata)[:,1]
    else:
        temp_pred = xgb1.predict_proba(testdata)[:,1]
        tp = test_pred
        test_pred = [i + j for i,j in zip(tp,temp_pred)]

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, gamma=0.2,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=478, n_jobs=None, nthread=-1, num_parallel_tree=None,
              random_state=None, reg_alpha=0.1, reg_lambda=None,
              scale_pos_weight=1, seed=2015, subsample=0.72, tree_method=None,
              validate_parameters=None, verbosity=None)

Model Report
Training Accuracy : 0.7822
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, gamma=0.2,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weig

In [25]:
print(plot_importance(xgb1))

NameError: name 'plot_importance' is not defined

In [27]:
for i,tp in enumerate(test_pred):
    test_pred[i] = tp/5
test_pred[:10]

[0.8370219230651855,
 0.31235227584838865,
 0.3248697519302368,
 0.7026053428649902,
 0.6164267539978028,
 0.4534462928771973,
 0.17022209167480468,
 0.15955185890197754,
 0.66219162940979,
 0.36257922649383545]

In [28]:
res_dict = {}
for user_id in range(30000,44999):
    if user_id == 43262:
        res_dict[user_id] =0
    elif user_id < 43262:
        res_dict[user_id] = test_pred[user_id - 30000]
    else:
        res_dict[user_id] = test_pred[user_id - 30001]

In [29]:
import csv

with open('result.csv','w') as csvfile:
    fieldnames = ['user_id','problem']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(30000,44999):
        if i not in res_dict.keys():
            writer.writerow({'user_id':i,'problem':0})
        else:
            writer.writerow({'user_id':i,'problem':res_dict[i]})

In [30]:
len(res_dict)

14999